# สัปดาห์ที่ 6: ระบบควบคุมเวิร์กโฟลว์ข้อมูล (Orchestration Systems) และ Modern Data Stack

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/witsarutsarai12-Academic/128-356-Big-Data/blob/main/BigData_Week12_Airflow_and_Modern_Data_Stack.ipynb)

![AirflowLogo](./images/AirflowLogo.png)

ในสัปดาห์นี้ เราจะเจาะลึกเกี่ยวกับการทำงานอัตโนมัติของ Data Pipeline ทำความรู้จักวิวัฒนาการตั้งแต่ยุคที่ไม่มีเครื่องมือควบคุม (Orchestrator) ไปจนถึงอดีตเครื่องมือดังในตลาด และทำไม **Apache Airflow** จึงกลายเป็นมาตรฐานของวงการ นอกจากนี้ เราจะพาไปทำความรู้จักกับเครื่องมือในยุค **Modern Data Stack** (dbt, Airbyte, Snowflake, Databricks) รวมถึงแนวคิดการทำ CI/CD (Continuous Integration / Continuous Deployment) สำหรับ Data Pipeline

---


## 1. Data Pipeline แบบเจาะลึก (Data Pipeline In-Depth)

**Data Pipeline** ไม่ใช่แค่การเขียนสคริปต์ดึงข้อมูลเพียงครั้งเดียว แต่เป็นระบบส่งต่อข้อมูลที่ต้องทำงานอย่างต่อเนื่องและเป็นอัตโนมัติ (Automated Ecosystem)

### วงจรชีวิตของข้อมูล (Data Lifecycle)
1. **Ingestion (Extract)**: การดึงข้อมูลจากแหล่งต้นทาง (Databases, Third-party APIs, File Servers, Logs)
2. **Storage (Load)**: การนำข้อมูลดิบไปจัดเก็บรวมไว้ที่ศูนย์กลาง เช่น Data Lake (S3, GCS) หรือ Data Warehouse
3. **Processing (Transform)**: การทำความสะอาด (Clean), กรอง (Filter), เชื่อมโยง (Join), และคำนวณสถิติ (Aggregate) ให้ได้ตารางข้อมูลที่พร้อมใช้
4. **Serving (Analytics/ML)**: นำข้อมูลที่ผ่านการประมวลผลไปสร้าง Dashboard (BI Tools) หรือใช้ Train Machine Learning Models

![Nanobanana Data Lifecycle](./images/nanobanana_data_lifecycle.png)
*(ภาพสี่ขั้นตอน Data Lifecycle พาทัวร์โรงงานข้อมูลโดย Nanobanana)*

### ความท้าทายของ Data Pipeline ในโลกจริง
เมื่อข้อมูลโตขึ้นและระบบซับซ้อนขึ้น เราจะพบว่าสคริปต์ธรรมดาเริ่มเอาไม่อยู่ เพราะ:
- **Dependency (ความสัมพันธ์ของงาน)**: งาน B ต้องรอ งาน A เสร็จก่อน ถ้า A ล้มเหลว B ต้องหยุดทำงานชั่วคราว
- **Retry & Recovery (การซ่อมแซมและคืนชีพ)**: ถ้าระบบต้นทางล่มชั่วคราว (เช่น API limit, Network ร่วง) ระบบควรจะ Retry ให้เอง ไม่ใช่พังแล้วแช่แข็ง
- **Monitoring & Alerting (การแจ้งเตือน)**: ถ้างานล้มเหลว Data Engineer ต้องรับรู้ผ่าน Slack/Email ทันที ไม่ใช่รอจน User มาบอกว่าตัวเลขผิด
- **Backfilling (การรันย้อนหลัง)**: ระบบสามารถรันงานของช่วง 1 ปีที่แล้วใหม่ได้ง่ายๆ เมื่อมีการแก้ Logic ของ Code โดยไม่ต้องเขียนสคริปต์ใหม่

---


## 2. ยุคก่อนมี Orchestrator: โลกที่ไร้ไม้คทาของวาทยกร

![Nanobanana Spaghetti Code](./images/nanobanana_spaghetti_cron.png)
*(ฝันร้ายของ Nanobanana เมื่อต้องสู้กับ Cron Jobs และ Spaghetti Code)*


ลองจินตนาการถึงองค์กรในยุคแรกเริ่มที่เพิ่งเริ่มทำ Data แนวคิดต่างๆ ยังใหม่มาก Data Pipeline มักเริ่มต้นจาก:
1. **Manual Run (พลังของสคริปต์บนเครื่องส่วนตัว)**: มี Analyst คนหนึ่ง (มักชื่อสมชาย) เขียน Python Script ไว้ในเครื่อง Laptop สมชายจะต้องตื่นมาตอน 8:00 น. เพื่อกดปุ่ม Run สคริปต์ชุดที่ 1 พอสคริปต์แรกรันเสร็จ ก็จะกดคลิกสคริปต์ชุดที่ 2 ถัดไป ถ้าระหว่างรัน อินเทอร์เน็ตพัง หรือสมชายลาป่วย ข้อมูลวันนั้นก็จะหายไปเลย!
2. **Trigger ทับ Trigger (มหากาพย์แห่ง Event แบบสปาเก็ตตี้)**: บริษัทพยายามอัปเกรดระบบ โดยเขียน Logic ไว้ว่า "เมื่อไฟล์ A ถูกดรอปลงในโฟลเดอร์ ให้เรียกใช้งาน Script B" และ "เมื่อ Script B จบ ให้เรียกใช้งาน Script C" ผลลัพธ์ที่ได้คือโค้ดพันกันยุ่งเหยิง (Spaghetti Code) ไม่มีใครรู้ภาพรวม (Global view) ว่า Pipeline จริงๆ ทำงานอย่างไร และถ้าพังกลางทางตรง B การจะสั่งเริ่มใหม่เฉพาะ C ก็กลายเป็นฝันร้าย

---


## 3. ยุคแห่งการตั้งเวลา (Cron Job) และข้อจำกัด

ในยุคถัดมา วิศวกรซอฟต์แวร์มักใช้ **Cron Job** (เครื่องมือตั้งเวลาการทำงานในระบบปฏิบัติการตระกูล Unix/Linux) เพื่อสั่งรันทาสก์ต่างๆ แบบตั้งเวลา เช่น:

`0 2 * * * /usr/bin/python3 run_extract.py` (ดึงข้อมูลตอนตี 2 ทุกวัน)
`0 3 * * * /usr/bin/python3 run_transform.py` (ประมวลผลข้อมูลตอนตี 3 ทุกวัน)

### ปัญหาของ Cron Job สำหรับ Data Pipeline อาชีพ
1. **ไม่รู้จักความสัมพันธ์ (No Dependency Management)** 
   - ตั้งไว้ว่ารัน Extract ตี 2, Transform ตี 3... แต่ถ้าวันนึง Extract ข้อมูลใหญ่มาก ใช้เวลาเกิน 1 ชั่วโมง (รันถึงตี 3 ครึ่ง) งาน Transform ที่ตั้งไว้ตี 3 ก็จะเริ่มรันไปพร้อมกัน ทำให้ประมวลผลข้อมูลที่ยังมาไม่ครบ (Data Inconsistency) หรือพัง (Crash)!
2. **ทำงานแบบปิดตา (No UI & Hard to Monitor)**
   - Cron ทำงานแบบ "เงียบพังเงียบ" (Silently fails) ถ้าคุณไม่เขียนโค้ดแจ้งเตือนเอง คุณจะไม่รู้เลยว่ามันพัง การจะดูว่าพังตรงไหนต้อง SSH เข้าไปไล่อ่านไฟล์ Log ยาวๆ เท่านั้น
3. **ขาดระบบจัดการความล้มเหลว (Rigid Failure Handling)**
   - หาก API ต้นทาง Timeout ชั่วคราว 1 วินาที Cron ไม่รู้จักคำว่า "รอ 5 นาทีแล้วลองใหม่" (Retry mechanism) ต้องจัดการเองในโค้ด
4. **วุ่นวายเมื่อต้องการประมวลผลย้อนหลัง (No Backfill support)**
   - หากต้องการรันข้อมูลย้อนหลัง 15 วัน ต้องเข้าไปแก้โค้ดเพื่อส่ง Parameter วันที่ใหม่ทีละวันๆ

---


## 4. วิวัฒนาการของเครื่องมือ Data Orchestrator (ก่อนยุค Airflow)

จากความวุ่นวายของวัฏจักร Cron โลกของ Data ผ่านวิวัฒนาการหลายยุค โดยแบ่งเป็นฝ่าย Enterprise ยุคเก่า และกลุ่ม Tech Giants:

### 4.1 ยุคเครื่องมือระดับลูกพี่ใหญ่ (Enterprise Tools - ยุค 1990s ถึง 2000s)
ก่อนยุค Cloud บรรดาธนาคารหรือองค์กรขนาดใหญ่มักใช้เครื่องมือระดับองค์กรที่มีราคาสูง เน้นการทำงานผ่านหน้าจอ UI แบบ **Drag-and-Drop (ลากวางกล่อง)** ยี่ห้อดังในยุคนี้ได้แก่:
1. **Informatica (PowerCenter)**: 
   - ตัวท็อปแห่งยุค ETL แบบ On-premise ราคามหาศาล เก่งมากเรื่องการ Transform ข้อมูลที่ซับซ้อนและการตรวจสอบคุณภาพข้อมูลระดับลึก
   - มีหน้าต่าง Workflow Manager เพื่อกดโยงเส้นลำดับกล่องการทำงาน
   - **ข้อจำกัด**: เป็นระบบปิด (Proprietary) ขยายตัวยาก (Scale ตามจำนวนเซิร์ฟเวอร์แบบเฉพาะเจาะจง) การเขียนโค้ด Custom เพิ่มทำได้ยากกว่ายุคปัจจุบัน
2. **Teradata**: 
   - ราชาแห่ง Data Warehouse ระดับองค์กร มาในรูปแบบ "ตู้เหล็กยักษ์" ผสมฮาร์ดแวร์และซอฟต์แวร์
   - มีเครื่องมือ (Utilities) ประจำตัวของมันที่ช่วยกวาดข้อมูลเข้าแบบขนาน (FastLoad, MultiLoad, BTEQ) ทรงพลังอย่างบ้าคลั่งในยุคที่ Cloud ยังไม่เกิด
   - **ข้อจำกัด**: ค่าใช้จ่ายแพงระยับ และมักจะผูกพันกับระบบของ Teradata เองเป็นหลัก รวมถึงไม่ได้เกิดมาเพื่อเจาะจงเป็นตัว Orchestrate กลางของเครื่องมืออื่น ๆ นอกระบบนัก
3. **IBM DataStage / SAP Data Services**: 
   - คู่แข่งของ Informatica ในระดับ Enterprise ที่เน้นการลากเส้นเชื่อมกล่องข้อมูลเข้าด้วยกัน

**สาเหตุที่เสื่อมความนิยม (สำหรับตัว Orchestrate กลาง):** เมื่อเข้าถึงยุค Big Data ที่ข้อมูลมีความหลากหลายและใหญ่ระดับ Petabyte เครื่องมือสำเร็จรูปราคาแพงที่รันบนเซิร์ฟเวอร์เดี่ยว (Scale-up) ไม่สามารถต้านทานไหว การเขียน Pipeline As Code จึงกลายเป็นทางเลือกใหม่

### 4.2 ยุค Open-source จากเหล่า Tech Giants (ยุค 2010s)
บรรดาบริษัทไอทีชั้นนำหนีจากโซลูชันเดิม พยายามสร้างระบบตัวเอง และปล่อยเป็น Open-source ให้ชุมชนใช้:
1. **Apache Oozie (เกิดจากยุค Hadoop)**: 
   - อดีตรุ่นพี่ผู้บุกเบิก เกิดมาเพื่อคุมคิวงานใน Hadoop Ecosystem (MapReduce, Hive)
   - **ปัญหา**: เขึยน Configuration ด้วยภาษา XML!! ซึ่งเยิ่นเย้อ ช้า ซับซ้อนมาก และขยายตัวยาก เมื่อโลกมุ่งหน้าทิ้ง Hadoop ตัว Oozie จึงตายตามไปด้วย
2. **Luigi (พัฒนาโดย Spotify)**: 
   - ยุโรปเปิดรับยุค *"Pipeline as Code"* ด้วยภาษา Python เป็นระบบที่จัดการ Dependency ได้ดี
   - **ปัญหา**: ไม่มี Scheduler (มีแค่ Trigger) และ UI ยังอ่อน จึงไม่ครบเครื่องรอบด้าน
3. **Azkaban (พัฒนาโดย LinkedIn)**: 
   - ระบบที่เน้น UI สวยงาม รองรับผู้ใช้หลายคนจัดการและกดรันได้ง่าย
   - **ปัญหา**: เขียนเป็นไฟล์ตั้งค่า (Properties file) ไม่ได้รับความยืดหยุ่น 100% จากการลงลึกถึงชั้น Source code แบบ Python

---


## 5. เจาะลึก Apache Airflow (The Engine of Data Pipelines)

เมื่อพบข้อจำกัดของบรรพบุรุษ **Maxime Beauchemin** (วิศวกรที่ Airbnb) จึงได้พัฒนา **Apache Airflow** ขึ้นมาในปี 2014 โดยรวมข้อดีคือการเขียนเป็นโค้ด Python (Pipeline as Python Code เหมือน Luigi) รวมกับระบบตั้งเวลา (Scheduler) ที่ทรงพลังและ UI ที่ครบเครื่องที่สุด จนกลายมาเป็นมาตรฐานอันดับหนึ่ง (Standard tool) ของวงการ Data Engineering

### แนวคิดหลัก: DAG (Directed Acyclic Graph)
แทนที่จะมองการรันโปรแกรมเป็นเส้นตรง Airflow รันงานเป็น **DAG (กราฟระบุทิศทางที่ไม่วนกลับ)**
- **Directed (มีทิศทาง)**: รู้ลำดับก่อนหลังชัดเจน งาน A ต้องเสร็จก่อน ถึงจะทำงาน B และ C ต่อได้
- **Acyclic (ไม่วนลูป)**: ลูกศรชี้ไปข้างหน้าเสมอ ไม่มีการวกกลับมาที่ Task เดิม เพื่อป้องกันไม่ให้เกิดลูปอนันต์ (Infinite loop)
- **Graph (กราฟ)**: วาดโครงสร้างงานแต่ละชิ้นเป็น Node (เรียกว่า Task) เชื่อมการไหลด้วยภาพ

การจัดความสัมพันธ์ (Dependencies) ใน Airflow ง่ายมาก เพียงเขียน:
`task_a >> [task_b, task_c] >> task_d`

![Nanobanana DAG Example](./images/airflow_dag_nanobanana.png)
*(ภาพตัวอย่าง DAG ขับร้องโดย Nanobanana ผู้อำนวยเพลง)*

---

### เจาะลึกสถาปัตยกรรม (Airflow Core Components)
Airflow ถูกออกแบบมาเป็น Distributed System มีส่วนประกอบสำคัญดังนี้:

1. **Web Server**: ส่วนที่คอยให้บริการ User Interface (UI) แก่ผู้ใช้ เพื่อให้เราสามารถกดเข้าไปดูสถานะการทำงาน (เขียว=สำเร็จ, แดง=ล้มเหลว) กดดู Logs หรือสั่ง Clear Task ให้รันใหม่ได้อย่างง่ายดาย
2. **Scheduler (สมองกล)**: คอยตรวจสอบและอ่านไฟล์ DAG ทั้งหมดอย่างต่อเนื่อง เพื่อดูว่ามี Task ไหนถึงเวลารันหรือยัง และ Task ก่อนหน้าเสร็จสมบูรณ์ไหม ถ้าเข้าเงื่อนไข จะส่งงานเข้าไปที่ Executor (ผ่าน Queue)
3. **Executor (ฝ่ายจัดการ)**: หน้าที่คือกำหนดว่า จะรันงานนั้นในเครื่องรูปแบบใด (เช่น `LocalExecutor` รันในเครื่องเดี่ยวพร้อมกันตามจำนวน CPU thread หรือ `KubernetesExecutor` ที่สร้าง Pod ใหม่ใน Cluster มารันโดยเฉพาะ)
4. **Worker (แรงงาน)**: ตัวเครื่อง/กระบวนการที่รับโค้ดและลงมือรันทำงานจริงๆ (Execution)
5. **Metadata Database**: ฐานข้อมูล (ปกติเป็น Postgres/MySQL) สำหรับเก็บ "สถานะทุกสรรพสิ่ง" (เช่น รันถึงไหนแล้ว, รันกี่ครั้ง, เวลาสำเร็จ, และ Connections ไปยังฐานข้อมูลปลายทาง)

![Nanobanana Airflow Architecture](./images/airflow_architecture_nanobanana.png)
*(สถาปัตยกรรม Airflow - นาโนบานานากำลังสั่งงานเหล่า Worker)*

---


## 6. สอนเขียน Airflow ทีละก้าว (Step-by-Step Tutorial)

ในการสร้าง Workflow 1 ตัว (เรียกว่า DAG) ใน Airflow นั้น เราจะต้องเขียนเป็นโค้ด Python โดยมีโครงสร้างหลักๆ 5 ส่วน ดังนี้:
1. **Import Modules**: เรียกใช้ไลบรารีของ Airflow และ Operators ที่จำเป็น
2. **Default Arguments**: ตั้งค่าพื้นฐานที่ใช้กับทุกงาน (Task) ใน DAG นี้ (เช่น การ Retry, เจ้าของ DAG)
3. **Instantiate a DAG**: สร้างตัวแปร DAG ขึ้นมา และกำหนด Schedule ว่าจะให้รันบ่อยแค่ไหน
4. **Tasks (Operators)**: สร้างงานแต่ละชิ้น โดยเลือกใช้ Operator ให้ตรงกับงาน (เช่น นำ BashOperator มารันคำสั่ง Terminal)
5. **Dependencies**: จับงานทั้งหมดมาโยงเส้นความสัมพันธ์กัน (วาดกราฟ)

### Step 1: Import ไลบรารีที่จำเป็น
ทุกไฟล์ใน Airflow ต้องมีการนำเข้าคลาส `DAG` ฝั่งของเนื้องาน เราจะใช้ `Operators` มาช่วยทำ ซึ่งมีหลายตัวมาก แต่วันนี้เราจะยกตัวอย่าง `BashOperator` (รันสคริปต์/คำสั่งบรรทัด) และ `PythonOperator` (รันฟังก์ชันไพธอน)

In [ ]:
!pip install apache-airflow

In [ ]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta

# ฟังก์ชันพิเศษสำหรับไว้เรียกใช้ด้วย PythonOperator
def print_hello():
    print("Hello from Airflow Python Operator!")
    return "Success"


### Step 2: กำหนด Default Arguments
มันคือ Dictionary ที่เก็บการตั้งค่าแบบรวมศูนย์ เพื่อให้ไม่ต้องพิมค่าย้ำๆ ในทุกงาน เช่น ถ้ารันพังให้ Retry ได้ 3 ครั้ง (ทิ้งช่วง 5 นาที)


In [ ]:
default_args = {
    'owner': 'data_engineering_student',    # ระบุชื่อผู้ดูแล DAG
    'depends_on_past': False,               # รันงานวันนี้ได้เลย ไม่ต้องรอของเมื่อวานให้สำเร็จก่อน
    'email_on_failure': False,              # ไม่ต้องส่งแจ้งเตือนถ้าพัง (ในระบบจริงมักเปิดเป็น True)
    'email_on_retry': False,                # ไม่ต้องเตือนถ้ากำลัง Retry
    'retries': 2,                           # จำนวนครั้งที่ยอมให้รันใหม่ถ้าล้มเหลว
    'retry_delay': timedelta(minutes=5),    # ระยะห่างในการ Retry แต่ละครั้ง
}


### Step 3: ประกาศโครงสร้างร่มใหญ่ของ DAG (Instantiate DAG)
เราจะต้องตั้งชื่อ (DAG_ID), ระบุวันเริ่มต้น และรอบการตั้งเวลา (Schedule Interval)

In [ ]:
with DAG(
    dag_id='my_first_airflow_tutorial',       # ชื่อของ DAG ห้ามซ้ำกับตัวอื่นในระบบ
    default_args=default_args,                # เรียกใช้การตั้งค่าจาก Step 2
    description='A simple tutorial DAG',      # คำอธิบายสั้นๆ (จะไปโชว์ใน UI)
    schedule_interval=timedelta(days=1),      # รันทุกๆ 1 วัน (สามารถเขียนเป็น Cron Expression ได้ เช่น '0 0 * * *')
    start_date=datetime(2023, 1, 1),          # วันเวลาเริ่มต้นที่ยอมให้ DAG ทำงานได้
    catchup=False                             # False = ไม่ต้องไล่รันย้อนหลังตั้งแต่วันที่ 1 ม.ค. ถึงปัจจุบัน (รันข้ามมาวันนี้เลย)
) as dag:
    # --- พื้นที่สำหรับใส่ Task ใน Step 4 และ 5 จะอยู่เยื้องใน block ถัดไป ---
    pass  # ใช้ pass ไปก่อนเพื่อไม่ให้ error

### Step 4 & 5: สร้างชิ้นงาน (Tasks) และร้อยเรียงทิศทาง (Dependencies)
เราจะเรียกใช้ Operators ออกมาสร้างเป็นตัวแปร (เปรียบเสมือน Worker แต่ละคนที่เรารับสมัครเข้ามา) ให้แน่ใจว่าได้ระบุ `task_id` ซึ่งเป็นชื่อประจำตึกของงานนั้น

**สุดท้าย (สำคัญที่สุด):** การกำหนด Dependency ด้วยเครื่องหมาย `>>`  (ลูกศรพุ่งไปข้างหน้า)

In [ ]:
with DAG(
    dag_id='my_first_airflow_tutorial', 
    default_args=default_args, 
    schedule_interval='@daily', 
    start_date=datetime(2023, 1, 1), 
    catchup=False
) as dag:

    # Task A: รันคำสั่ง Print ธรรมดาผ่าน Terminal
    job_1 = BashOperator(
        task_id='start_greeting',
        bash_command='echo "Starting my awesome pipeline!"'
    )

    # Task B: สั่ง Sleep นับถอยหลัง 
    job_2 = BashOperator(
        task_id='wait_for_10_seconds',
        bash_command='sleep 10'
    )

    # Task C: รันฟังก์ชันไพธอนที่เตรียมไว้ด้านบน
    job_3 = PythonOperator(
        task_id='run_my_python_code',
        python_callable=print_hello
    )

    # Task 4: ขอลองสร้างอีกตัวให้มารันขนานกับ Job 3
    job_4_parallel = BashOperator(
        task_id='i_am_running_parallel',
        bash_command='echo "This runs at the same time as job 3!"'
    )

    # ------ ประกอบส่วน Dependency ------
    # ความสัมพันธ์: ให้ 1 รันเสร็จก่อน ค่อยตามด้วย 2
    # จากนั้น 3 และ 4 รันพร้อมกันเพราะต่อท้าย 2 เหมือนกัน
    
    job_1 >> job_2 >> [job_3, job_4_parallel]
    
    # เพียงเท่านี้ เราก็จะได้กราฟ 1-2-<(3,4)> อย่างสมบูรณ์!

เมื่อนำซอร์สโค้ดนี้ไปวางลงในโฟลเดอร์รันของ Airflow ตัว **Scheduler** ของ Airflow จะตรวจเจอไฟล์นี้โดยอัตโนมัติ ภายใน 1 นาที มันจะไปแสดงผลขึ้นที่ฝั่งหน้าจอ **Web Server UI** เป็นตารางสวยงาม ให้เรากด Monitor ได้ทันที!

---
### Step 6: จะลองเล่น Airflow ได้อย่างไร? (Installation & Free Alternatives)
การจะเซ็ตอัปเครื่องเซิร์ฟเวอร์ Airflow เต็มสูบแบบ Distributed System ต้องใช้ความพยายามสูงมาก (ต้องตั้ง Database, Workers, Webserver) แต่สำหรับผู้เริ่มต้น มี 3 ทางเลือกยอดฮิต:

**ทางเลือกที่ 1: ติดตั้งแบบรวบรัดบนเครื่องตัวเอง (Local Docker) - แนะนำที่สุด**
Airflow มีไฟล์ `docker-compose.yaml` แจกฟรี เพียงแค่ในเครื่องคุณมีโปรแกรม Docker Desktop ก็ทำตามรันคำสั่งนี้ได้เลย:
```bash
curl -LfO 'https://airflow.apache.org/docs/apache-airflow/stable/docker-compose.yaml'
docker-compose up -d
```
จากนั้นเข้า Web Browser ไปที่ `http://localhost:8080` (Username/Password: airflow) ก็จะเจอกับหน้าจอ UI ให้กดเล่นได้เลย

**ทางเลือกที่ 2: ติดตั้งด้วย Python PIP (เหมาะกับทดสอบเล่นๆ แบบ Standalone)**
```bash
pip install apache-airflow
airflow standalone
```
คำสั่งนี้จะบีบอัดทุกอย่าง (Web Server, Scheduler, Database แบบย่อ) ลงมาอยู่ใน Terminal บรรทัดเดียว เหมาะสำหรับผู้ริเริ่มเขียน DAG เบื้องต้น

**ทางเลือกที่ 3: ใช้บริการ Cloud (Managed Service - ไม่ต้องลงโปรแกรมเอง)**
บริษัทส่วนใหญ่มักเช่า Airflow สำเร็จรูป ไม่นิยมติดตั้งเองด้วยคอนเทนเนอร์ เช่น
- **Astronomer (Astro)**: แพลตฟอร์มจากบริษัทของกลุ่มผู้สร้าง Airflow มี **Free Trial ให้ทดลองประทับตราฟรี** ล็อกอินและ Deploy Pipeline ขึ้น Cloud ได้เลย เหมาะสำหรับผู้เริ่มต้นที่ไม่อยากเปิด Docker ในคอม
- **Amazon MWAA / Google Cloud Composer**: Airflow ที่อินทิเกรตอย่างสมบูรณ์แบบกับ AWS / Google Cloud (มีค่าใช้จ่าย)

---


### โบนัส: รัน Airflow UI บน Google Colab (วิธีเสถียรกว่าเดิม)

บน Colab เราเปิด `localhost:8080` ตรงๆ ไม่ได้ จึงต้องทำ tunnel ออกมาภายนอก
ในเวอร์ชันนี้เปลี่ยนจาก **localtunnel** เป็น **cloudflared** ซึ่งเสถียรกว่าและไม่ต้องกรอกรหัสผ่านหน้าเว็บคั่นกลาง

**ขั้นตอนการรันบน Colab (รันทีละ Cell):**
1. ติดตั้ง Airflow ด้วย constraints ให้ตรง Python ของ Colab
2. ติดตั้ง `cloudflared`
3. สตาร์ต `airflow standalone` และรอให้ Web Server พร้อม
4. เปิด tunnel และคลิก URL ที่ได้ (`https://...trycloudflare.com`)


In [ ]:
# 1) ติดตั้ง Airflow (ล้างแพ็กเกจเก่าก่อนเพื่อกันชนกันระหว่าง Airflow 2.x/3.x)
import subprocess
import sys

py_ver = f"{sys.version_info.major}.{sys.version_info.minor}"
airflow_ver = "2.10.5"
constraints_url = f"https://raw.githubusercontent.com/apache/airflow/constraints-{airflow_ver}/constraints-{py_ver}.txt"

print(f"Installing Airflow {airflow_ver} for Python {py_ver}")
print(f"Constraints: {constraints_url}")

# เคลียร์แพ็กเกจ Airflow ที่อาจค้างจากคนละ major version (โดยเฉพาะ Airflow 3 split packages)
%pip uninstall -y -q apache-airflow apache-airflow-core apache-airflow-task-sdk apache-airflow-providers-standard

# ติดตั้งใหม่แบบ clean
%pip install -q --no-cache-dir "apache-airflow=={airflow_ver}" --constraint "{constraints_url}"

# Colab hotfix: บาง runtime มี typing_extensions เก่า ทำให้ pydantic_core import Sentinel ไม่ได้
%pip uninstall -y -q typing_extensions
%pip install -q --no-cache-dir --force-reinstall "typing_extensions>=4.12.2"

import importlib
import airflow
import typing_extensions as te
importlib.reload(te)
print("Airflow:", airflow.__version__, airflow.__file__)
print("Kernel typing_extensions:", te.__file__, getattr(te, "__version__", "unknown"), "Sentinel:", hasattr(te, "Sentinel"))

# ตรวจใน subprocess ด้วย (airflow command จะรันเป็น subprocess)
subprocess.run(
    [
        sys.executable,
        "-c",
        "import airflow, typing_extensions as te; print('Subprocess Airflow:', airflow.__version__, airflow.__file__); print('Subprocess typing_extensions:', te.__file__, getattr(te,'__version__','unknown'), 'Sentinel:', hasattr(te,'Sentinel'))",
    ],
    check=True,
)


In [ ]:
# 2) ติดตั้ง Cloudflared เพื่อ tunnel พอร์ต 8080
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
!cloudflared --version


In [ ]:
import json
import os
import shutil
import subprocess
import sys
import time
import urllib.request

import airflow

AIRFLOW_HOME = "/content/airflow"
os.environ["AIRFLOW_HOME"] = AIRFLOW_HOME
os.environ["AIRFLOW__CORE__LOAD_EXAMPLES"] = "False"
os.environ["AIRFLOW__WEBSERVER__WORKERS"] = "1"

# Force default XCom backend ให้ตรงกับ Airflow major version (กัน config เก่าชนกัน)
airflow_major = int(airflow.__version__.split(".")[0])
if airflow_major >= 3:
    os.environ["AIRFLOW__CORE__XCOM_BACKEND"] = "airflow.sdk.execution_time.xcom.BaseXCom"
else:
    os.environ["AIRFLOW__CORE__XCOM_BACKEND"] = "airflow.models.xcom.BaseXCom"

print("Airflow version:", airflow.__version__)
print("Using XCom backend:", os.environ["AIRFLOW__CORE__XCOM_BACKEND"])

# กัน process เก่าค้าง
os.system("pkill -f 'airflow webserver' >/dev/null 2>&1")
os.system("pkill -f 'airflow scheduler' >/dev/null 2>&1")
os.system("pkill -f 'airflow standalone' >/dev/null 2>&1")
os.system("pkill -f 'cloudflared tunnel' >/dev/null 2>&1")
time.sleep(2)

# ล้าง AIRFLOW_HOME เก่าที่อาจค้าง state พัง
if os.path.exists(AIRFLOW_HOME):
    shutil.rmtree(AIRFLOW_HOME)
os.makedirs(AIRFLOW_HOME, exist_ok=True)

# Fallback patch: ถ้า runtime ยังเจอ typing_extensions เก่า ให้เติม Sentinel ตอนเริ่ม interpreter
fix_dir = "/content/pyfix"
os.makedirs(fix_dir, exist_ok=True)
with open(f"{fix_dir}/sitecustomize.py", "w") as f:
    f.write(
        "import typing_extensions as te\n"
        "if (not hasattr(te, 'Sentinel')) and hasattr(te, '_Sentinel'):\n"
        "    te.Sentinel = te._Sentinel\n"
    )

env = os.environ.copy()
env["PYTHONPATH"] = fix_dir + (":" + env["PYTHONPATH"] if env.get("PYTHONPATH") else "")

# sanity check in subprocess ก่อนรัน standalone
subprocess.run(
    [
        sys.executable,
        "-c",
        "import os, airflow; print('Preflight Airflow:', airflow.__version__, airflow.__file__); print('Preflight XCom backend env:', os.getenv('AIRFLOW__CORE__XCOM_BACKEND'))",
    ],
    check=True,
    env=env,
)

print("Starting airflow standalone...")
airflow_log = "/content/airflow_standalone.log"
with open(airflow_log, "w") as logf:
    proc = subprocess.Popen(["airflow", "standalone"], stdout=logf, stderr=subprocess.STDOUT, env=env)

# รอให้ Airflow เปิด health endpoint (นานขึ้นสำหรับ Colab ที่ช้า)
ready = False
for i in range(420):
    if proc.poll() is not None:
        print("Airflow process exited before becoming healthy.")
        break
    try:
        urllib.request.urlopen("http://127.0.0.1:8080/health", timeout=2)
        ready = True
        print("Airflow webserver is ready on :8080")
        break
    except Exception:
        if i % 20 == 0:
            print(f"Waiting for Airflow... {i}s")
        time.sleep(1)

if not ready:
    print("\n===== airflow_standalone.log (tail) =====")
    if os.path.exists(airflow_log):
        with open(airflow_log) as f:
            lines = f.readlines()
        print("".join(lines[-150:]))
    raise RuntimeError("Airflow did not become ready. See log tail above.")

# ดึง username/password ที่ airflow generate ให้
password = None
username = "admin"
new_pwd_file = f"{AIRFLOW_HOME}/simple_auth_manager_passwords.json.generated"
old_pwd_file = f"{AIRFLOW_HOME}/standalone_admin_password.txt"

if os.path.exists(new_pwd_file):
    with open(new_pwd_file) as f:
        auth_data = json.load(f)
        username = list(auth_data.keys())[0]
        password = auth_data[username]
elif os.path.exists(old_pwd_file):
    with open(old_pwd_file) as f:
        content = f.read().strip()
        password = content.split("Password:")[-1].strip() if "Password:" in content else content

print(f"Username: {username}")
if password:
    print(f"Password: {password}")
else:
    print("Password file not found yet. You can inspect /content/airflow_standalone.log")


In [ ]:
import os
import re
import subprocess
import time

# 4) เปิด Cloudflared tunnel และแสดง URL สำหรับเข้า Airflow UI
os.system("pkill -f 'cloudflared tunnel' >/dev/null 2>&1")

tunnel_log = "/content/cloudflared.log"
with open(tunnel_log, "w") as logf:
    subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://127.0.0.1:8080", "--no-autoupdate"],
        stdout=logf,
        stderr=subprocess.STDOUT,
    )

public_url = None
for _ in range(60):
    time.sleep(1)
    if os.path.exists(tunnel_log):
        with open(tunnel_log) as f:
            logs = f.read()
        m = re.search(r"https://[-a-zA-Z0-9]+\.trycloudflare\.com", logs)
        if m:
            public_url = m.group(0)
            break

if public_url:
    print("Open this Airflow UI URL:")
    print(public_url)
    print("If the URL expires, rerun this cell to create a new tunnel.")
else:
    print("Tunnel URL not found yet. Run: !tail -n 80 /content/cloudflared.log")


## 7. ยุคของ Modern Data Stack (MDS)

เมื่อเทคโนโลยี Cloud ก้าวหน้าขึ้น องค์กรต้องการสร้าง Data Pipeline ที่รวดเร็วกว่าเดิม โดยใช้เครื่องมือสำเร็จรูปมากขึ้น ทิศทางของ Pipeline เปลี่ยนจากกรอบ **ETL (Extract -> Transform -> Load)** เดิมแบบ On-premise ที่กระบวนการแปลง (Transform) ต้องใช้เครื่องประมวลผลขนาดใหญ่ตรงกลาง มาสู่แนวคิด **ELT (Extract -> Load -> Transform)**

**หลักการของ ELT**: *"ก็ดูดข้อมูลดิบทั้งหมด (E) ไปกองไว้ใน Cloud Data Warehouse ยักษ์ๆ เลย (L) แล้วค่อยใช้พลังคิวรี (Query) ที่มหาศาลของ Warehouse นั้นๆ ในการแปลงข้อมูล (T)"* 

![Modern Data Stack Architecture](./images/modern-data-stack.png)
*(ภาพจำลองโครงสร้าง Modern Data Stack ที่มีเครื่องมือใหม่ๆ เข้ามาแทนที่ - ที่มา: Metabase)*

Modern Data Stack (MDS) จึงประกอบด้วยเครื่องมือชูโรง ดังนี้:

### 7.1. ฝ่ายดึงข้อมูล (The "Extract & Load" Phase): Airbyte / Fivetran
- ภาระการเขียนสคริปต์ดึง API ของฐานข้อมูลนับ 100 แหล่ง เป็นเรื่องน่าเบื่อและซ้ำซาก (ต้องคอยแก้เวลา API เค้าอัปเดตเวอร์ชัน)
- **Airbyte** (Open-source) และ **Fivetran** (Commercial) คือเครื่องมือสำเร็จรูปที่ทำหน้าที่ดึงข้อมูลอย่างเดียว (EL) มี Connector สำเร็จรูปมหาศาล (Facebook Ads, Salesforce, MongoDB ฯลฯ) แค่กดคลิกผ่าน UI ข้อมูลก็จะไหลเข้า Warehouse ปลายทางอย่างต่อเนื่อง

### 7.2. ฐานทัพหลัก (The Storage & Compute Engine): Snowflake / Databricks
- **Snowflake (Cloud Data Warehouse)**: 
  - เปลี่ยนวงการด้วยสถาปัตยกรรมที่แยก **Storage** กับ **Compute** ออกจากกันอย่างเด็ดขาด อยากเก็บข้อมูลมหาศาลก็จ่ายค่าเช่ารายเดือนถูกๆ พอๆ กับ S3 (อิงตามพื้นที่) อยากคิวรีรีพอร์ตหนักๆ สิ้นเดือนก็ค่อยเปิดโหนด Compute แรงๆ มาวิเคราะห์ จบแล้วก็ปิดทิ้งได้ 
  - โดดเด่นด้านการใช้งานแบบ SQL และเหมาะใช้ทำ Data Management เต็มรูปแบบ
- **Databricks (Data Lakehouse)**: 
  - สร้างจากทีมผู้ก่อตั้ง Apache Spark โดยพยายามประสานจุดเด่นของ มาสู่แนวคิด "Lakehouse"
  - ใช้เป็นหัวใจหลักได้ทั้งงานวิเคราะห์ SQL ทั่วไป, งาน Streaming ระดับมิลลิวินาที และงานจัดเตรียมข้อมูลสร้างโมเดล Machine Learning แบบครบวงจร

### 7.3. ฝ่ายแปลงข้อมูล (The "Transform" Phase): dbt (data build tool)
- จากเดิมที่ Data Engineer ต้องเขียนโค้ดซับซ้อน หรือ Analyst ต้องทำ Scheduled Query เป็นร้อยๆ ตัวทับซ้อนกันใน Warehouse
- **dbt (data build tool)** คือฮีโร่ที่เข้ามาทำหน้าที่ตระกูล "T" ใน ELT โดยยอมให้เราเขียนแค่คำสั่ง `SELECT` พื้นฐาน แต่อัดฉีดความเป็น Software Engineering เข้าไปลึกสุดขั้ว
- **ฟีเจอร์เด่น:**
  1. โค้ดถูกผูกกับ Version Control (Git) ใครแก้เมื่อไหร่ทีมรู้หมด
  2. วาดกราฟความสัมพันธ์ (Dependency/Data Lineage) ของตารางข้อมูลทั้งหมดให้แบบอัตโนมัติ รู้อดีต รู้อนาคตว่าถ้าทำตาราง A พัง ตาราง B ที่รอใช้จะพังตาม
  3. มีระบบ Data Testing แบบ Built-in ช่วยดักจับ Error เชิงโครงสร้างข้อมูลไว้ก่อนรันลง Production จริง (เช่น คอลัมน์ User_ID ห้ามเป็นค่า Null ห้ามมีค่าซ้ำ)

---


## 8. แถม: CI/CD Pipeline สำหรับ Data Engineering (Automation Beyond Airflow)

![Nanobanana CI/CD](./images/nanobanana_cicd.png)
*(Nanobanana คุมเครื่องจักรสายพาน CI/CD - ทุกอย่างต้องอัตโนมัติ!)*


เราใช้ Airflow จัดการความสัมพันธ์ของ "Data"... 
แต่ในการทำงานของวิศวกรข้อมูล โค้ดที่พวกเขาเขียนขึ้นล่ะ ใครเป็นคนตรวจสอบ และฝังลงไปบน Server? คำตอบคือ **CI/CD Pipelines (Continuous Integration / Continuous Deployment)**

แนวคิดนี้ถูกส่งผ่านมาจาก Software Engineering (DevOps) สู่ Data Engineering:
- **CI (การรวมโค้ดต่อเนื่อง)**: เมื่อมีนักพัฒนาเขียนโค้ดใหม่ (เช่น เพิ่ม DAG ใน Airflow หรือแก้ dbt model) และทำการ Push ขึ้น Git ระบบจะนำโค้ดนั้นมารัน **Tests** อัตโนมัติ เพื่อเช็คการแตกหัก (Syntax Error / Logic Error)
- **CD (การส่งมอบต่อเนื่อง)**: เมื่อผ่านการเทสทั้้งหมด โค้ดใหม่จะถูก นำไปวาง (Deploy) บน Production Server อัตโนมัติโดยที่มนุษย์ไม่ต้องอัพโหลดไฟล์เอง

### เครื่องมือชื่อดังในตลาด
1. **Jenkins (รุ่นเก๋า ยืดหยุ่นสูงสุด)**: 
   - เครื่องมือขวัญใจทีม DevOps ยุคบุกเบิก เป็น Automation Server ที่เขียน Pipeline เชิงสคริปต์ได้ทุกอย่าง (อยากให้รัน Shell, ยิง API ทำได้หมด)
2. **Argo CD (สาย Kubernetes)**:
   - ปัจจุบัน Infrastructures มักวิ่งอยู่บนระบบตู้คอนเทนเนอร์ (Kubernetes) 
   - Argo CD โดดเด่นด้วยโมเดล *GitOps* คือ "โค้ดใน Git เขียนอย่างไร, สภาพแวดล้อมจริงบนเซิร์ฟเวอร์ต้องเป็นอย่างนั้นเป๊ะๆ" ทันทีที่มีการแก้โค้ด Kubernetes จะดึง (Pull) การเปลี่ยนแปลงไปปรับทันทีแบบไร้รอยต่อ
3. **Terraform (Infrastructure as Code - IaC)**:
   - แทนที่คุณจะเข้าหน้าเว็บ AWS/GCP ไปกดคลิกสร้าง Server 10 เครื่อง (รัน Airflow), สร้าง Database 1 ตัว, สร้าง Cloud Storage... คุณเขียนทั้งหมดเป็นไฟล์ **Terraform Code** (`.tf`) 
   - พอรัน `terraform apply` มันจะไปคุยกับ Cloud provider สร้าง Server ปลั๊กอิน Network จัดสรร Database ให้ทั้งหมดตามที่พิมพ์ไว้ (ถ้าระบบพัง ก็แค่ใช้โค้ดชุดนี้รันใหม่ ได้บ้านหลังใหม่หน้าตาเหมือนเดิม 100% ภายใน 5 นาที!)

**ข้อคิด:** Airflow เป็นไม้คทาของ Data แต่ CI/CD & Terraform คือไม้คทาแห่ง Infrastructure!

---


## 9. Workshop: สร้าง Data Pipeline ของจริงด้วยชุดข้อมูลจริง

📝 **โจทย์:** เพื่อให้เห็นภาพการทำงานจริง เราจะสร้าง Pipeline กรองข้อมูลผู้โดยสารเรือ Titanic ยอดฮิต โดยมี 3 ขั้นตอนที่ต้อง**รอให้งานก่อนหน้าทำเสร็จก่อน**:
1. **Download Data**: โหลดไฟล์ CSV จากอินเทอร์เน็ต
2. **Process Data**: รันสคริปต์ Python แยกต่างหาก (Standalone Job) เพื่อลบแถวที่ข้อมูลหาย และสรุปยอดผู้รอดชีวิต
3. **Archive**: ย้ายไฟล์ที่ประมวลผลเสร็จแล้วไปเก็บในโฟลเดอร์ Archive

*(แผนผังการทำงาน: `Download` ➜ `Process` ➜ `Archive`)*

### ขั้นตอนที่ 1: สร้างโฟลเดอร์สำหรับจำลองระบบการทำงาน
ก่อนอื่น เราจำลองว่าเรากำลังทำงานอยู่ในเครื่อง Server จริงๆ

In [ ]:
import os

# สร้างโฟลเดอร์จำลอง
os.makedirs("my_data_pipeline/raw_data", exist_ok=True)
os.makedirs("my_data_pipeline/processed_data", exist_ok=True)
os.makedirs("my_data_pipeline/archive", exist_ok=True)

print("✅ สร้างโฟลเดอร์สำหรับรับส่งข้อมูลสำเร็จเป้าหมายแล้ว!")

### ขั้นตอนที่ 2: เขียน Python Job (Standalone) ให้ลูกน้อง (Worker) เอาไปรัน
เราจะไม่เอา Code จัดการข้อมูล (Data Transformation) ไปยัดไว้ในไฟล์ Airflow DAG ตรงๆ แต่จะเขียนแยกเป็นไฟล์ `clean_titanic.py` ให้ Airflow เรียกใช้อีกที (Best Practice)

In [ ]:
%%writefile my_data_pipeline/clean_titanic.py
import pandas as pd
import os

def run_cleaning():
    input_file = "my_data_pipeline/raw_data/titanic.csv"
    output_file = "my_data_pipeline/processed_data/titanic_cleaned.csv"
    
    # 1. เช็คก่อนว่ามีไฟล์ให้ทำไหม (ถ้าไม่มีแปลว่า Task ก่อนหน้าพังหรือไม่ได้รัน)
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"❌ ไม่พบไฟล์ {input_file} (Task ดาวน์โหลดอาจจะพัง!)")

    print("✅ พบไฟล์ข้อมูลดิบ กำลังเริ่มกระบวนการ Clean...")
    
    # 2. อ่านข้อมูล
    df = pd.read_csv(input_file)
    original_rows = len(df)

    # 3. คลีนข้อมูล (Drop แถวที่มีค่า Null ในคอลัมน์ Age)
    df_clean = df.dropna(subset=["Age"])
    
    # 4. เซฟไฟล์ที่เสร็จแล้วลงโฟลเดอร์ปลายทาง
    df_clean.to_csv(output_file, index=False)
    
    print(f"📊 สรุปผลการ Clean:")
    print(f"   - ข้อมูลต้นทาง: {original_rows} แถว")
    print(f"   - ข้อมูลหลัง Clean (ตัด Age ว่างทิ้ง): {len(df_clean)} แถว")
    print(f"✅ เซฟไฟล์ผลลัพธ์ไปที่ {output_file} เรียบร้อยแล้ว")

if __name__ == "__main__":
    run_cleaning()

### ขั้นตอนที่ 3: เอาเชือกมาผูกงานด้วย Airflow DAG (`titanic_pipeline.py`)
ทีนี้ก็ถึงคราวพระเอกของเรา (Airflow) ออกโรงจัดระเบียบ 3 งานนี้ให้ทำงานต่อคิวกันอย่างสมบูรณ์แบบ

In [ ]:
%%writefile my_data_pipeline/titanic_pipeline.py
from airflow import DAG
from airflow.operators.bash import BashOperator
from datetime import datetime, timedelta

default_args = {
    "owner": "data_student",
    "retries": 2, # ถ้าพังให้ลองใหม่ 2 ครั้ง
    "retry_delay": timedelta(minutes=1),
}

with DAG(
    dag_id="titanic_data_processing_pipeline",
    default_args=default_args,
    schedule_interval="@daily", 
    start_date=datetime(2023, 1, 1),
    catchup=False
) as dag:

    # 📌 งานที่ 1: โหลดไฟล์ CSV ตำนานเรือไททานิกด้วย cURL ชิลๆ
    download_task = BashOperator(
        task_id="download_raw_data",
        bash_command="curl -sL https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv -o my_data_pipeline/raw_data/titanic.csv"
    )

    # 📌 งานที่ 2: รันสคริปต์ Python ตัวเก่งของเรา (ต้องรอโหลกเสร็จนะไม่งั้นพัง!)
    process_task = BashOperator(
        task_id="clean_data_with_python",
        bash_command="python3 my_data_pipeline/clean_titanic.py"
    )

    # 📌 งานที่ 3: ยกไฟล์ต้นฉบับไปเก็บที่ Archive จะได้ไม่รกโฟลเดอร์ (ต้องรอ Process เสร็จก่อน)
    archive_task = BashOperator(
        task_id="archive_raw_data",
        bash_command="mv my_data_pipeline/raw_data/titanic.csv my_data_pipeline/archive/titanic_$(date +%Y%m%d_%H%M%S).csv"
    )

    # 🌟 พระเอกอยู่ตรงนี้: จัดเรียงคิวการทำงาน (Dependency Wiring)
    # ถ้า download_task พัง หรือโหลดไม่เสร็จ -> process_task จะไม่เริ่มเด็ดขาด!
    download_task >> process_task >> archive_task

print("✅ สร้างไฟล์ Airflow DAG เรียบร้อยแล้ว!")

### 🌟 ขั้นตอนที่ 4: สิ่งที่จะเกิดขึ้นเมื่อ Airflow จัดการ Pipeline นี้

1. **`download_raw_data` เริ่มทำงาน:** ยิง API โหลดไฟล์จาก Github 
   - ถ้าระหว่างโหลดเน็ตหลุด -> ระบบจะขึ้นสถานะ **Failed** สีแดง และพักรอ 1 นาที (ตามที่เราตั้ง `retries` ไว้) แล้วพยายามโหลดใหม่
2. **`clean_data_with_python` รอจังหวะ:** งานจำศีลรอจนกว่ากล่องแรกจะขึ้นสถานะ **Success** สีเขียว
3. พอไฟเขียวปุ๊บ ระบบสั่งรัน `python3 clean_titanic.py` ทันที ข้อมูลดิบถูกแปรสภาพเป็นข้อมูลพร้อมวิเคราะห์อยู่ใน `processed_data/`
4. **`archive_raw_data` เก็บกวาด:** เมื่อ Python กรอกข้อมูลเสร็จ งานที่ 3 จะย้ายไฟล์ต้นฉบับไปเข้ากรุ `archive/` ป้องกันไม่ให้การรันในวันพรุ่งนี้มีไฟล์ทับซ้อนกัน

และนี่แหละครับ คือพลังของ **"Orchestration"**! 🎉

---
